## Support Vector Machines: Fit and evaluate a model

Using the Titanic dataset from [this](https://www.kaggle.com/c/titanic/overview) Kaggle competition.

In this section, we will fit and evaluate a simple Support Vector Machines model.

### Read in Data

<!-- <img src="img/svm/CV.png" width="700">
<img src="img/svm/Cross-Val.png" width="500"> -->
<table><tr><td><img src="img/svm/CV.png" width="1000"></td><td><img src="img/svm/Cross-Val.png" width="1000"></td></tr></table>

In [8]:
import joblib
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

feature_columns = ['Pclass', 'Gender', 'Age', 'Fare', 'Family_count', 'Cabin_index']
label_column = ['Survival']

tr_features = pd.read_csv('dataset/train_dataset.csv', usecols=feature_columns)
tr_labels = pd.read_csv('dataset/train_dataset.csv', usecols=label_column)

### Hyperparameter tuning

<img src="img/svm/c.png" width="600">

In [9]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [10]:
svc = SVC()
# kernels -> linear, radial basis function
parameters = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10]
}

cv = GridSearchCV(svc, parameters, cv=5)
cv.fit(tr_features, tr_labels.values.ravel())

print_results(cv)

BEST PARAMS: {'C': 0.1, 'kernel': 'linear'}

0.796 (+/-0.115) for {'C': 0.1, 'kernel': 'linear'}
0.654 (+/-0.06) for {'C': 0.1, 'kernel': 'rbf'}
0.796 (+/-0.115) for {'C': 1, 'kernel': 'linear'}
0.661 (+/-0.048) for {'C': 1, 'kernel': 'rbf'}
0.796 (+/-0.115) for {'C': 10, 'kernel': 'linear'}
0.684 (+/-0.07) for {'C': 10, 'kernel': 'rbf'}


In [11]:
cv.best_estimator_

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

### Write out pickled model

In [12]:
joblib.dump(cv.best_estimator_, 'models/svm_model.pkl')

['models/svm_model.pkl']

### Load the SVM model

In [13]:
#Loading the saved model
pipe = joblib.load('models/svm_model.pkl')

### Predict

In [15]:
# test data to predict
pr = pd.read_csv('dataset/test_dataset.csv')
# remove the label column
pred_cols = list(pr.columns.values)[:-1]
# apply the whole pipeline to data
pred = pd.Series(pipe.predict(pr[pred_cols]))

# set of predictions
print("Prediction")
print(pred[25:40].tolist())

# select label column
label_cols = list(pr.columns.values)[-1]

# corresponding actual labels
print("\nActual labels")
print(pr[label_cols][20:35].tolist())

Prediction
[1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]

Actual labels
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1]
